In [1]:
import pandas as pd
import numpy as np
import time
import turicreate as tc

In [2]:
alunos = pd.read_csv('../data/alunos.csv')
matriculas = pd.read_csv('../data/matriculas.csv')

In [3]:
# etapa 1 da organizando dos cursos
matriculas['cursos'] = matriculas['cursos'].apply(lambda x: [int(i) for i in x.split('|')])
matriculas.head(2).set_index('alunoId')['cursos'].apply(pd.Series).reset_index()

# etapa 2: criando um dataframe com os campos alunoId, cursoId, e matricula_count
pd.melt(matriculas.head(2).set_index('alunoId')['cursos'].apply(pd.Series).reset_index(), 
             id_vars=['alunoId'],
             value_name='cursos') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['alunoId', 'cursos']) \
    .agg({'cursos': 'count'}) \
    .rename(columns={'cursos': 'matricula_count'}) \
    .reset_index() \
    .rename(columns={'cursos': 'cursoId'})

,alunoId,cursoId,matricula_count
0,1076254,123465.0,1
1,1076254,123507.0,1
2,1076254,124321.0,1
3,1076254,920083.0,1
4,1076254,920089.0,1
5,1076254,920090.0,1
6,1268548,122912.0,1
7,1268548,124321.0,1


In [4]:
data = pd.melt(matriculas.set_index('alunoId')['cursos'].apply(pd.Series).reset_index(), 
             id_vars=['alunoId'],
             value_name='cursos') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['alunoId', 'cursos']) \
    .agg({'cursos': 'count'}) \
    .rename(columns={'cursos': 'matricula_count'}) \
    .reset_index() \
    .rename(columns={'cursos': 'cursoId'})
data['cursoId'] = data['cursoId'].astype(np.int64)

In [5]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

NameError: name 'train_test_split' is not defined

In [ ]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [48]:
train_data

alunoId,cursoId,matricula_count
1269542,920081,1
1739070,920086,1
1702662,125,1
1714775,92,1
1827626,92,1
1786530,920089,1
1763201,920085,1
1786647,920089,1
1732092,920084,1
1806219,124740,1


In [ ]:
test_data

In [ ]:

user_id = 'alunoId'
item_id = 'cursoId'
users_to_recommend = list(matriculas[user_id])
n_rec = 10 # múmero de ítens para receomendar
n_display = 30 #número de alunos para mostrar

cosine_model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    similarity_type='cosine')

pearson_model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    similarity_type='pearson')

cosine_recomm = cosine_model.recommend(users=users_to_recommend, k=n_rec)
cosine_recomm.print_rows(n_display)

pearson_recomm = pearson_model.recommend(users=users_to_recommend, k=n_rec)
pearson_recomm.print_rows(n_display)


In [25]:
cosine_recomm = cosine_model.recommend(users=users_to_recommend, k=n_rec)
cosine_recomm.print_rows(n_display)

recommendations finished on 1000/7987 queries. users per second: 34867.5

recommendations finished on 2000/7987 queries. users per second: 42661.2

recommendations finished on 3000/7987 queries. users per second: 40286.8

recommendations finished on 4000/7987 queries. users per second: 34156.5

recommendations finished on 5000/7987 queries. users per second: 28284.2

recommendations finished on 6000/7987 queries. users per second: 27049.4

recommendations finished on 7000/7987 queries. users per second: 25197.1

+---------+---------+---------------------+------+
| alunoId | cursoId |        score        | rank |
+---------+---------+---------------------+------+
| 1076254 |  920085 | 0.16347451011339822 |  1   |
| 1076254 |  920091 | 0.15428065260251364 |  2   |
| 1076254 |  920084 | 0.13212081789970398 |  3   |
| 1076254 |  920087 | 0.12941795587539673 |  4   |
| 1076254 |  920082 | 0.12900487581888834 |  5   |
| 1076254 |  920080 |  0.1280336081981659 |  6   |
| 1076254 |  920081 | 0.12535054485003153 |  7   |
| 1076254 |  920094 | 0.12057096759478252 |  8   |
| 1076254 |  920088 | 0.10784764091173808 |  9   |
| 1076254 |  920086 | 0.10758921504020691 |  10  |
| 1268548 |  123564 | 0.19999998807907104 |  1   |
| 1268548 |  124683 | 0.19999998807907104 |  2   |
| 1268548 |  124517 | 0.19999998807907104 |  3   |
| 1268548 |  124669 |  0.141421377658844  |  4   |
| 1268548 |  124496 |  0.141421377658844  |  5   |
| 1268548 |  123777 |  0.141421377658844  |  6   |
| 1268548 |  124467 |  0.141421

In [51]:
pearson_recomm = pearson_model.recommend(users=users_to_recommend, k=n_rec)
pearson_recomm.print_rows(n_display)

recommendations finished on 1000/7987 queries. users per second: 55732

recommendations finished on 2000/7987 queries. users per second: 69170.6

recommendations finished on 3000/7987 queries. users per second: 75481.2

recommendations finished on 4000/7987 queries. users per second: 78706.1

recommendations finished on 5000/7987 queries. users per second: 81017.6

recommendations finished on 6000/7987 queries. users per second: 83850.4

+---------+---------+---------------------+------+
| alunoId | cursoId |        score        | rank |
+---------+---------+---------------------+------+
| 1076254 |  920081 |         1.0         |  1   |
| 1076254 |  920091 |  0.908485856905158  |  2   |
| 1076254 |    92   |  0.9076539101497504 |  3   |
| 1076254 |  920085 |  0.6913477537437605 |  4   |
| 1076254 |   124   |  0.651414309484193  |  5   |
| 1076254 |  920083 |  0.591514143094842  |  6   |
| 1076254 |  920080 |  0.5515806988352745 |  7   |
| 1076254 |  920082 |  0.5282861896838602 |  8   |
| 1076254 |  920084 |  0.5141430948419301 |  9   |
| 1076254 |  920087 | 0.48336106489184694 |  10  |
| 1268548 |  920081 |         1.0         |  1   |
| 1268548 |  920091 |  0.908485856905158  |  2   |
| 1268548 |    92   |  0.9076539101497504 |  3   |
| 1268548 |  920089 |  0.7579034941763727 |  4   |
| 1268548 |  920085 |  0.6913477537437605 |  5   |
| 1268548 |   124   |  0.651414309484193  |  6   |
| 1268548 |  920083 |  0.591514

recommendations finished on 7000/7987 queries. users per second: 84429.9

In [56]:
tc.recommender.util.compare_models(test_data, [cosine_model, pearson_model], model_names=["cosine", "pearson"], exclude_known_for_precision_recall=False)

PROGRESS: Evaluate model cosine


recommendations finished on 1000/3200 queries. users per second: 39243.4

recommendations finished on 2000/3200 queries. users per second: 37207.9

recommendations finished on 3000/3200 queries. users per second: 38926.7


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.1506250000000002 | 0.09937574404761897 |
|   2    | 0.12671875000000005 | 0.16130357142857132 |
|   3    |       0.11125       | 0.21140029761904772 |
|   4    |     0.102421875     | 0.26102008928571435 |
|   5    | 0.09243750000000006 | 0.29290550595238124 |
|   6    | 0.08567708333333326 |  0.3234627976190474 |
|   7    | 0.07991071428571433 |  0.351202380952381  |
|   8    | 0.07468749999999996 |     0.374234375     |
|   9    | 0.07013888888888883 |  0.3950364583333334 |
|   10   | 0.06593750000000002 |  0.4122247023809523 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model pearson


recommendations finished on 1000/3200 queries. users per second: 50563.8

recommendations finished on 2000/3200 queries. users per second: 38797.3

recommendations finished on 3000/3200 queries. users per second: 35000.1


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.08687500000000008 | 0.055248139880952396 |
|   2    |      0.08296875     | 0.10381659226190475  |
|   3    | 0.08385416666666672 | 0.18944159226190482  |
|   4    | 0.07937499999999992 | 0.23306584821428594  |
|   5    | 0.07656250000000005 |  0.272502604166667   |
|   6    | 0.07354166666666657 |  0.3309401041666669  |
|   7    |  0.070982142857143  | 0.36535801091269804  |
|   8    | 0.06812499999999994 | 0.39393737599206313  |
|   9    |  0.0670138888888889 |  0.4304578373015874  |
|   10   | 0.06612500000000004 | 0.46773127480158716  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]



[{'precision_recall_by_user': Columns:
  	alunoId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 57600
  
  Data:
  +---------+--------+---------------------+--------+-------+
  | alunoId | cutoff |      precision      | recall | count |
  +---------+--------+---------------------+--------+-------+
  | 1269542 |   1    |         1.0         |  1.0   |   1   |
  | 1269542 |   2    |         0.5         |  1.0   |   1   |
  | 1269542 |   3    |  0.3333333333333333 |  1.0   |   1   |
  | 1269542 |   4    |         0.25        |  1.0   |   1   |
  | 1269542 |   5    |         0.2         |  1.0   |   1   |
  | 1269542 |   6    | 0.16666666666666666 |  1.0   |   1   |
  | 1269542 |   7    | 0.14285714285714285 |  1.0   |   1   |
  | 1269542 |   8    |        0.125        |  1.0   |   1   |
  | 1269542 |   9    |  0.1111111111111111 |  1.0   |   1   |
  | 1269542 |   10   |         0.1         |  1.0   |   1   |
  +---------+--------+---------------------+------